Importing necessary liabraries

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split


In [ ]:
# Load and preprocess the image dataset

def load_images(image_dir):
    images = []
    labels = []
    for file_name in os.listdir(image_dir):
        img = cv2.imread(os.path.join(image_dir, file_name))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
        img = cv2.resize(img, (224, 224))  # Resize to fit CNN input
        images.append(img)
        labels.append(extract_text(file_name))  # Assuming text is extracted from image filename
    images = np.array(images)
    labels = np.array(labels)
    images = images.reshape(-1, 224, 224, 1)  # Reshaping for CNN input
    images = images.astype('float32') / 255.0  # Normalize images
    return images, labels

# Assuming the labels are embedded within the filenames, for example: 'cat_001.jpg'
def extract_text(file_name):
    # Extract the label (text) from the filename
    label = file_name.split('_')[0]  # e.g., 'cat' from 'cat_001.jpg'
    return label

# Split dataset into training and testing sets
def split_dataset(images, labels, test_size=0.2):
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

# Build the CNN model
def build_cnn_model():
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')  # Assuming 10 classes for simplicity
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Visualizing some images from the dataset
def visualize_images(images, labels, num_images=5):
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(images[i].reshape(224, 224), cmap='gray')
        plt.title(f"Label: {labels[i]}")
        plt.axis('off')
    plt.show()

# Training the CNN model
def train_model(model, X_train, y_train, epochs=10, batch_size=32):
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    return history

# Evaluate the model on the test set
def evaluate_model(model, X_test, y_test):
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_acc * 100:.2f}%")
    return test_loss, test_acc

# Plot training and validation accuracy/loss
def plot_history(history):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()


image_dir = "path/to/images"  # Replace with your image directory path
images, labels = load_images(image_dir)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = split_dataset(images, labels)

# Build the CNN model
cnn_model = build_cnn_model()

# Visualize some sample images from the dataset
visualize_images(X_train, y_train, num_images=5)

# Train the model
history = train_model(cnn_model, X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model on the test set
test_loss, test_acc = evaluate_model(cnn_model, X_test, y_test)

# Plot the training history
plot_history(history)

# Predict on new test images
predictions = cnn_model.predict(X_test)

# Visualize some predictions
def visualize_predictions(predictions, X_test, y_test, num_images=5):
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(1, num_images, i+1)
        plt.imshow(X_test[i].reshape(224, 224), cmap='gray')
        predicted_label = np.argmax(predictions[i])
        plt.title(f"Pred: {predicted_label}, True: {y_test[i]}")
        plt.axis('off')
    plt.show()

# Visualize some predictions
visualize_predictions(predictions, X_test, y_test, num_images=5)
